## Load Libraries


In [ ]:
import os
import pandas as pd
from statsforecast import StatsForecast
from statsforecast.utils import ConformalIntervals
import datetime
from utilsforecast.plotting import plot_series
from statistics import mean


from statsforecast.models import (
    HoltWinters,
    CrostonClassic as Croston, 
    HistoricAverage,
    DynamicOptimizedTheta,
    SeasonalNaive,
    AutoARIMA,
    AutoRegressive,
    AutoETS,
    AutoTBATS,
    MSTL,
    Holt
)



import plotly.express as px

## Load the Data

Reformat the data


In [ ]:
ts = pd.read_csv("data/data.csv")
ts["ds"] = pd.to_datetime(ts["ds"])
ts = ts.sort_values("ds")
ts = ts[["unique_id", "ds", "y"]]

os.environ['NIXTLA_ID_AS_COL'] = '1'

ts.head()

Subset for the last 25 months:

In [ ]:
end = ts["ds"].max()
start = end - datetime.timedelta(hours = 24 * 31 * 25)
ts = ts[ts["ds"] >= start]

## Set the Backtesting

Define the forecasting models:


In [ ]:
# Instantiate models with hourly seasonality
auto_arima = AutoARIMA(alias="model_AutoARIMA")
s_naive = SeasonalNaive(season_length=24,alias="model_SeasonalNaive")
theta =  DynamicOptimizedTheta(season_length= 24, 
alias="model_DynamicOptimizedTheta")

# Instantiate models with hourly and weekly seasonality
mstl1 = MSTL(season_length=[24, 24 * 7], 
		    trend_forecaster=AutoARIMA(),
		    alias="model_MSTL_ARIMA_trend")


mstl2 = MSTL(season_length=[24, 24 * 7], 
		    trend_forecaster=Holt(),
		    alias="model_MSTL_Holt_trend")

mstl3 = MSTL(
    season_length=[24, 24 * 7], 
    trend_forecaster=  AutoRegressive(lags= list(range(1, 24)), include_mean=True),
    alias="model_MSTL_AR_trend_1" 
)


In [ ]:
stats_models = [auto_arima, s_naive, theta, mstl1, mstl2]
# stats_models = [mstl2]

sf = StatsForecast(
    models = stats_models, 
    freq = 'h', 
    n_jobs = -1
)

In [ ]:
h = 72
step_size = 24
partitons = 2
n_windows=5
method = "conformal_distribution"
intervals = ConformalIntervals(h=72, n_windows=n_windows, method = method)
levels = [95]

In [ ]:
bkt_df = sf.cross_validation(
    df = ts,
    h = h,
    step_size = step_size,
    n_windows =  partitons,
    prediction_intervals = intervals,
    level = levels
  )

In [ ]:
bkt_df.head()

In [ ]:
model_label = [str(s) for s in stats_models]  
model_name = [type(s).__name__ for s in stats_models]  

models_mapping = pd.DataFrame({"model_label": model_label, 
            "model_name": model_name})


d1 = pd.melt(bkt_df, id_vars= ["unique_id", "ds", "cutoff"], 
value_vars= model_label, var_name = "model_label" , value_name = "forecast")
d2 = pd.melt(bkt_df, id_vars= ["unique_id", "ds", "cutoff"], 
value_vars= lower, var_name = "model_label" , value_name = "lower")
d2["model_label"] = d2["model_label"].str.replace("-lo-95", "")
d3 = pd.melt(bkt_df, id_vars= ["unique_id", "ds", "cutoff"], 
value_vars= upper, var_name = "model_label", value_name = "upper")
d3["model_label"] = d3["model_label"].str.replace("-hi-95", "")

bkt_long = (
    d1
.merge(right = d2, how = "left", on = ["unique_id", "ds", "cutoff", "model_label"])
.merge(right = d3, how = "left", on = ["unique_id", "ds", "cutoff", "model_label"])
.merge(right =  models_mapping, how = "left", on = ["model_label"])
.merge(right =  ts, how = "left", on = ["unique_id", "ds"])
)
bkt_long.head()

In [ ]:
def mape(y, yhat):
    mape = mean(abs(y - yhat)/ y) 
    return mape

def rmse(y, yhat):
    rmse = (mean((y - yhat) ** 2 )) ** 0.5
    return rmse

def coverage(y, lower, upper):
    coverage = sum((y <= upper) & (y >= lower)) / len(y)
    return coverage


def score(df):
    mape_score = mape(y = df["y"], yhat = df["forecast"])
    rmse_score = rmse(y = df["y"], yhat = df["forecast"])
    coverage_score = coverage(y = df["y"], lower = df["lower"], upper = df["upper"])
    cols = ["mape","rmse", "coverage"]
    d = pd.Series([mape_score, rmse_score,  coverage_score], index=cols)

    return d

score_df = bkt_long.groupby(["unique_id", "model_label", "model_name", "cutoff"]).apply(score).reset_index()

score_df.head()

In [ ]:
leaderboard = score_df.groupby(["unique_id", "model_label", "model_name"]).agg({"mape": "mean", "rmse": "mean", "coverage": "mean"}).reset_index()

leaderboard.sort_values(by = ["mape"])

In [ ]:
fig = px.box(score_df, x="model_label", y="rmse", color="model_label")
fig.update_traces(boxpoints = 'all', jitter = 0.3, pointpos = -1.8, showlegend = False)

fig.update_layout(
    title="Error Distribution",
    xaxis_title="Model",
    yaxis_title="RMSE",
    font=dict(family="Arial", size=14, color="black")
)

fig.show()

## Logging the Results with MLflow


In [ ]:
cutoff = bkt_long["cutoff"].unique()
partitions_mapping = pd.DataFrame({"cutoff": cutoff, "partition": range(1, len(cutoff) + 1)})

partitions_mapping

score_df = score_df.merge(partitions_mapping, how = "left", on = ["cutoff"])
score_df

In [ ]:
import mlflow

experiment_name = "stats_forecast"

mlflow_path = "file:///mlruns"

tags = {"h": h,
"step_size": step_size,
"partitions": partitons,
"intervals_type": "ConformalIntervals",
"intervals_h": h,
"intervals_n_windows": n_windows,
"intervals_method": "conformal_distribution",
"levels": levels }



try:
    mlflow.create_experiment(name = experiment_name,
                            artifact_location= mlflow_path,
                            tags = tags)
    meta = mlflow.get_experiment_by_name(experiment_name)
    print(f"Set a new experiment {experiment_name}")
    print("Pulling the metadata")
except:
    print(f"Experiment {experiment_name} exists, pulling the metadata")
    meta = mlflow.get_experiment_by_name(experiment_name)

In [ ]:
run_time = datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S")
for index, row in score_df.iterrows():
    run_name = row["model_label"] + "-" + run_time 
    with mlflow.start_run(experiment_id = meta.experiment_id, 
                run_name = run_name,
                tags = {"type": "backtesting",
                "partition": row["partition"], 
                "unique_id": row["unique_id"],
                "model_label": row["model_label"],
                "model_name": row["model_name"],
                "run_name": run_name}) as run:
        model_params = {
            "model_name": row["model_name"],
            "model_label": row["model_label"],
            "partition": row["partition"]
        }
        mlflow.log_params(model_params)
        mlflow.log_metric("mape", row["mape"])
        mlflow.log_metric("rmse", row["rmse"])
        mlflow.log_metric("coverage", row["coverage"])